# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 24 2023 10:24AM,261619,18,SEQ-9412344,Seqirus,Released
1,Apr 24 2023 10:22AM,261624,19,9411297,"Innogenix, LLC",Released
2,Apr 24 2023 10:20AM,261623,10,LUPLUP0025,"Lupin Research, Inc.",Released
3,Apr 24 2023 10:17AM,261622,20,REL1-5849,Reliable 1 Laboratories LLC,Released
4,Apr 24 2023 10:17AM,261622,20,REL1-5851,Reliable 1 Laboratories LLC,Released
5,Apr 24 2023 10:17AM,261622,20,REL1-5852,Reliable 1 Laboratories LLC,Released
6,Apr 24 2023 10:17AM,261622,20,REL1-5853,Reliable 1 Laboratories LLC,Released
7,Apr 24 2023 10:17AM,261622,20,REL1-5854,Reliable 1 Laboratories LLC,Released
8,Apr 24 2023 10:17AM,261622,20,REL1-5855,Reliable 1 Laboratories LLC,Released
9,Apr 24 2023 10:17AM,261622,20,REL1-5856,Reliable 1 Laboratories LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,261620,Released,2
10,261621,Released,1
11,261622,Released,50
12,261623,Released,1
13,261624,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261620,NaN,2.0
261621,NaN,1.0
261622,NaN,50.0
261623,NaN,1.0
261624,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261596,13.0,6.0
261602,0.0,1.0
261612,0.0,29.0
261615,18.0,2.0
261616,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261596,13,6
261602,0,1
261612,0,29
261615,18,2
261616,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261596,13,6
1,261602,0,1
2,261612,0,29
3,261615,18,2
4,261616,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261596,13,6
1,261602,,1
2,261612,,29
3,261615,18,2
4,261616,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 24 2023 10:24AM,261619,18,Seqirus
1,Apr 24 2023 10:22AM,261624,19,"Innogenix, LLC"
2,Apr 24 2023 10:20AM,261623,10,"Lupin Research, Inc."
3,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC
53,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc."
54,Apr 24 2023 10:06AM,261620,15,"Brookfield Pharmaceuticals, LLC"
56,Apr 24 2023 10:02AM,261618,15,"Alliance Pharma, Inc."
84,Apr 24 2023 9:34AM,261616,10,"Citieffe, Inc."
85,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC"
114,Apr 24 2023 8:48AM,261602,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 24 2023 10:24AM,261619,18,Seqirus,,1
1,Apr 24 2023 10:22AM,261624,19,"Innogenix, LLC",,1
2,Apr 24 2023 10:20AM,261623,10,"Lupin Research, Inc.",,1
3,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC,,50
4,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc.",,1
5,Apr 24 2023 10:06AM,261620,15,"Brookfield Pharmaceuticals, LLC",,2
6,Apr 24 2023 10:02AM,261618,15,"Alliance Pharma, Inc.",,28
7,Apr 24 2023 9:34AM,261616,10,"Citieffe, Inc.",,1
8,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC",,29
9,Apr 24 2023 8:48AM,261602,16,TASA USA Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 10:24AM,261619,18,Seqirus,1,
1,Apr 24 2023 10:22AM,261624,19,"Innogenix, LLC",1,
2,Apr 24 2023 10:20AM,261623,10,"Lupin Research, Inc.",1,
3,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC,50,
4,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc.",1,
5,Apr 24 2023 10:06AM,261620,15,"Brookfield Pharmaceuticals, LLC",2,
6,Apr 24 2023 10:02AM,261618,15,"Alliance Pharma, Inc.",28,
7,Apr 24 2023 9:34AM,261616,10,"Citieffe, Inc.",1,
8,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC",29,
9,Apr 24 2023 8:48AM,261602,16,TASA USA Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 10:24AM,261619,18,Seqirus,1,
1,Apr 24 2023 10:22AM,261624,19,"Innogenix, LLC",1,
2,Apr 24 2023 10:20AM,261623,10,"Lupin Research, Inc.",1,
3,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC,50,
4,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 10:24AM,261619,18,Seqirus,1,NaN
1,Apr 24 2023 10:22AM,261624,19,"Innogenix, LLC",1,NaN
2,Apr 24 2023 10:20AM,261623,10,"Lupin Research, Inc.",1,NaN
3,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC,50,NaN
4,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 10:24AM,261619,18,Seqirus,1,0.0
1,Apr 24 2023 10:22AM,261624,19,"Innogenix, LLC",1,0.0
2,Apr 24 2023 10:20AM,261623,10,"Lupin Research, Inc.",1,0.0
3,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC,50,0.0
4,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,784851,31,0.0
15,784853,32,18.0
16,261602,1,0.0
18,261619,1,0.0
19,523220,7,13.0
20,523243,51,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,784851,31,0.0
1,15,784853,32,18.0
2,16,261602,1,0.0
3,18,261619,1,0.0
4,19,523220,7,13.0
5,20,523243,51,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,31,0.0
1,15,32,18.0
2,16,1,0.0
3,18,1,0.0
4,19,7,13.0
5,20,51,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,15,Released,32.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Executing,0.0,18.0,0.0,0.0,13.0,0.0
Released,31.0,32.0,1.0,1.0,7.0,51.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Executing,0.0,18.0,0.0,0.0,13.0,0.0
1,Released,31.0,32.0,1.0,1.0,7.0,51.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Executing,0.0,18.0,0.0,0.0,13.0,0.0
1,Released,31.0,32.0,1.0,1.0,7.0,51.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()